In [1]:
from langchain.llms.bedrock import Bedrock
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
import boto3
import botocore
config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 0}
)


bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)

file_path = (r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\Bryckle\Dallas, TX Olympus, Master Lease Agreement, Brillio LLC.pdf")

loader = PyPDFLoader(file_path)
docs=loader.load()


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=0)
texts = text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(documents=texts,embedding=embeddings)

template = """
    You will be given a query, Your task is to find an answer or give information about the query with respect to the document by performing a similarity search.
    Consider the following conditions,
    - If query is a topic, look for information or statements or sentences which are related to the query in the document.
    - The answer does not need to be specifically related, it can be loosely related as well.
    - If you find any statements that are directly related to the query, then explain those statements in easy or layman terms, so someone with no expertise in that field can understand.
    - If query is a question, understand the context of query and then look for similar statements or sentences in the document which also have the same context.
    - It is fine if you cannot find any directly related statements in the document. You can look for sentences with similar meaning and can also return loosely related answers.
    - Elaborate the answer as much as you can.
    - If you cannot find a relevant answer, then perform a similarity search on all statements which have a similar meaning to the query, and the document.

        Instructions:
            If the question is about
                - Is there rent abatement granted? Describe the conditions based on which it is calculated and how long is the abatement period in a few sentences? If none, state "There is no mention of any abatement."
                - Does the lease reference a "lock in period" within the intital term which implies that the agreement can be terminated with notice after lock in period, describe its terms and conditions." If none, state "The agreement is binding until expiration date"
                - If applicable, what is the percentage of shared expenses the tenant is responsible for, based on their leased space compared to the total leasable area? if there is no percentage expressed then state " The document is silent on this"
                - Explain how operating expenses ,CAM or service fee will be calculated by the landlord, presented to tenant and when is it payable. State the amount, if it is mentioned and specify if it is monthly or by size such as per square foot or square meter or per workstation. if there is none, state " The document is silent on this".
                - Landlord typically incurs a property insurance cost for the property within which the lease space exists and charges a proportionate share to the tenant. Is property insurance cost for the tenants share included in base rent or operating expenses? If yes, state " Property insurance costs are included in base rent" or "Property insurance costs are included in operating expenses" as applicable. If not, explain what is the amount the tenant has to pay, how will it be calculated, how will it be presented to tenant and when will it have to be paid? If there is absolutely no mention of this cost then state " The document is silent on this"
                - Landlord typically incurs a property tax for the property within which the lease space exists and charges a proportionate share to the tenant. Is property tax cost for the tenants share included in base rent or operating expenses? If yes, state " Property taxes are included in base rent" or "Property taxes are included in operating expenses" as applicable. If not, what is the amount the tenant has to pay, how will it be calculated, how will it be presented to tenant and when will it have to be paid? If there is absolutely no mention of this cost then state " The document is silent on this"
                - Is there any variable rent payable payable directly to the landlord based on lessee's consumption of utlities such as electricity, airconditioning,water and internet ? How is it calculated and payable? If there is none, state " The document is silent on this"
                - Is there any reference to paying percentage of sales/revenue as additional rent to lessor? If yes, what are the specifications? f there is none, state " The document is silent on this"
                - Is there a one time set up/onboarding or activation fee charged and what are the details? f there is none, state " The document is silent on this"
                - If there an amount of the tenant improvement allowance/construction allowance specified in the agreement, explain any conditions or requirements associated with the use? If there is none, state " The document is silent on this"
                - What is the penalty for rent payment after the due date , grace period and notification requirements? f there is none, state " The document is silent on this"
                - Is there any reference to residual value gaurantee ie after all rent payments, a guarantee of value of the underlying asset returned to the lessor at the end of the lease will be at least a specified amount? if yes , what is the amount? If there is none, state " The document is silent on this"
                - Is there an option for renewal or extension after the initial term and what are the conditions- describe in a few sentences? Include the process to exercise renewal option, give notices  and deadline for its execution. If there is none state " The document is silent on this"
                - Does the lessee have option of purchasing the space and under what conditions? If none, state " There is no mention of a purchase option."
                - Does the lease state any provision of exclusive use for lessee. If yes, capture the scope, period, limitations and enforcement details. If not state " The document is silent on this"
                - What are the building hours of operation. If none are mentioned, state " The document is silent on this"
                - Summarize all clauses that references alterations or improvements permitted, restrictions, performance of work limitations and enforcment.  If none are mentioned, state " The document is silent on this"
                - Summarize all clauses that reference landlords rights to access and entry as well as any limitations on it. If none are mentioned, state " The document is silent on this"
                - Summarize the holdover clause. Identify and extract all information on the duration and conditions under which a holdover fee may apply, the amount and consequences associated with holdover.  If none are mentioned, state " The document is silent on this"
                - Are there any terms and conditions of the co-tenancy specified in the document and the consequences or implications of changes in co-tenancy?f there is none, state " The document is silent on this"
                - What does the agreement specify regarding the obligation of the tenant for regular upkeep/regular maintenance/AMC's of certain systems or facilities within the space by the tenant? Describe all. If there is none, state " The document is silent on this"
                - What is the responsibility of tenant to obtain utilities directly from providers, list all specific ones and any conditions. If there is none, state " The document is silent on this"
                - Summarize the clauses on utilities provided by the landlord as part of base rent or opearting expenses and what happens if there are any overages.  If there is none, state " The document is silent on this"
                - If the tenant does not wish to renew is there a notice period required for ending the initial term or does it automatically end? Explain, what are the conditions and procedures? f there is none, state " The document is silent on this"
                - Does the tenant have the right to terminate the lease early, within the binding period of the agreement? if yes what conditions can trigger it, what is the notice required to do so and penalties if applicable? If tenant does not have any option to terminate state " Tenant cannot terminate this lease early"
                - Summarize all the clauses on subordination and attornment- listing both separately.  If there is none, state " The document is silent on this"
                - Summarize all the clauses on destruction of property, what happens in the event of fire or any such similar event of casualty.   If there is none, state " The document is silent on this"
                - Summarize in simple english the clause on estoppel, scope, preclusion of contradictory claims and enforceability. If there is none, state " The document is silent on this"
                - Summarize all clauses in simple english on scope of indemnification, indemnification trigger events, limitations and exclusions, notice and defense obligations, survival of indemnity obligations and enforceability and remedies.  If there is none, state " The document is silent on this"
                - Summarize all clauses insimple english on emninent domain and condemnation. If there is none state, " The document is silent on this"
            so if the question is about any one of above if the exact answer is not in the given document means simply state the information in the end of the question
{context}
{question}
"""

In [11]:
retriever = db.as_retriever(search_type='similarity', search_kwargs={"k": 3})
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 30000})
qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])
chain_type_kwargs = { "prompt": qa_prompt, "verbose": False }
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)

In [17]:
question="What is the frequency and specific date of rent payment?"
result = qa.run(question)
print(result)

 Based on the lease agreement, the following details are provided regarding rent payment:

Frequency of Rent Payment:
- Basic Rent is payable monthly, in advance on the first day of each month. 
- However, if Tenant's rent payments are delinquent for 2 consecutive months, Landlord can require that Basic Rent be paid quarterly in advance.

Specific Date of Rent Payment:
- Basic Rent for the first partial month is due on the Commencement Date. 
- For all subsequent months, Basic Rent is due on the first day of each month.

So in summary, Basic Rent is payable monthly in advance on the 1st day of each month. But if there are delinquencies, Landlord can require quarterly payments instead.
